## Importing the Libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import os

## Importing the Libraries

In [2]:
folder_path = './data/WorldCup_Stats'

# Get the all csv files into a list
csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]

In [3]:
# Combine all DataFrames into crick_df
crick_df = pd.concat([pd.read_csv(os.path.join(folder_path, file)) for file in csv_files], ignore_index=True)

In [4]:
# crick_df.to_csv('./data/crick_df.csv')

## Exploration and Cleaning the crick_df

In [5]:
# Check is there any duplicate values

print(crick_df.duplicated().sum())

0


In [6]:
crick_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 528 entries, 0 to 527
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0.1     528 non-null    int64  
 1   Unnamed: 0       528 non-null    int64  
 2   date             364 non-null    object 
 3   venue            528 non-null    object 
 4   match_category   528 non-null    object 
 5   team_1           528 non-null    object 
 6   team_2           528 non-null    object 
 7   team_1_runs      518 non-null    float64
 8   team_1_wickets   518 non-null    float64
 9   team_2_runs      513 non-null    float64
 10  team_2_wickets   513 non-null    float64
 11  result           528 non-null    object 
 12  pom              510 non-null    object 
 13  best_batters     250 non-null    object 
 14  best_bowlers     250 non-null    object 
 15  commentary_line  83 non-null     object 
 16  world_cup_year   528 non-null    int64  
 17  host_country    

## Handling Missing Values

Let's consider the `team_1_runs` column.

In [7]:
null_recs = crick_df[crick_df['team_1_runs'].isnull()]
null_recs.head()

,Unnamed: 0.1,Unnamed: 0,date,venue,match_category,team_1,team_2,team_1_runs,team_1_wickets,team_2_runs,team_2_wickets,result,pom,best_batters,best_bowlers,commentary_line,world_cup_year,host_country
11,11,7,NaN,The Oval,League-Match,SL,WI,NaN,NaN,NaN,NaN,Match abandoned without a ball bowled,NaN,NaN,NaN,NaN,1979,England
24,7,13,1996-02-25,Colombo (RPS),League-Match,SL,WI,NaN,NaN,NaN,NaN,Sri Lanka won (walkover without a ball bowled),NaN,NaN,NaN,NaN,1996,Sri Lanka
37,20,4,NaN,Colombo (RPS),League-Match,SL,AUS,NaN,NaN,NaN,NaN,Sri Lanka won (walkover without a ball bowled),NaN,NaN,NaN,NaN,1996,Sri Lanka
66,9,10,2019-06-07,Bristol,League-Match,Pakistan,Sri Lanka,NaN,NaN,NaN,NaN,Match abandoned without a ball bowled,NaN,NaN,NaN,NaN,2019,England
68,11,17,2019-06-13,Nottingham,League-Match,India,New Zealand,NaN,NaN,NaN,NaN,Match abandoned without a ball bowled,NaN,NaN,NaN,NaN,2019,England


Considering null records in `crick_df['team_1_runs']`,


*   All of the records in the columns `team_1_runs`, `team_2_runs`, `team_1_wickets`, and `team_2_wickets` are missing values.

*   However, the result column indicates that some of these matches were marked as **"Match abandoned."**

Therefore, we cannot remove these records immediately, as we plan to add a new column in the future to indicate the final match status (e.g., whether the match was played or abandoned).


In [8]:
# Let's add 0 values for the values given as match abandoned in result column

# Identify abandoned rows
abandoned_matches = crick_df['result'].str.contains('Match abandoned without a ball bowled', case=False, na=False)

# Update team_1_runs, team_2_runs, team_1_wickets, team_2_wickets to 0 for abandoned matches
crick_df.loc[abandoned_matches, ['team_1_runs', 'team_2_runs', 'team_1_wickets', 'team_2_wickets']] = 0


In [9]:
#Verify Chancges
crick_df[crick_df['result'].str.contains('Match abandoned without a ball bowled', case=False, na=False)].head()

,Unnamed: 0.1,Unnamed: 0,date,venue,match_category,team_1,team_2,team_1_runs,team_1_wickets,team_2_runs,team_2_wickets,result,pom,best_batters,best_bowlers,commentary_line,world_cup_year,host_country
11,11,7,NaN,The Oval,League-Match,SL,WI,0.0,0.0,0.0,0.0,Match abandoned without a ball bowled,NaN,NaN,NaN,NaN,1979,England
66,9,10,2019-06-07,Bristol,League-Match,Pakistan,Sri Lanka,0.0,0.0,0.0,0.0,Match abandoned without a ball bowled,NaN,NaN,NaN,NaN,2019,England
68,11,17,2019-06-13,Nottingham,League-Match,India,New Zealand,0.0,0.0,0.0,0.0,Match abandoned without a ball bowled,NaN,NaN,NaN,NaN,2019,England
93,36,15,2019-06-11,Bristol,League-Match,Bangladesh,Sri Lanka,0.0,0.0,0.0,0.0,Match abandoned without a ball bowled,NaN,NaN,NaN,NaN,2019,England
440,48,10,NaN,Brisbane,League-Match,Australia,Bangladesh,0.0,0.0,0.0,0.0,Match abandoned without a ball bowled,NaN,NaN,NaN,NaN,2015,Australia


Let's consider the `team_2_runs` column.

In [10]:
null_recs = crick_df[crick_df['team_2_runs'].isnull()]
null_recs.head()

,Unnamed: 0.1,Unnamed: 0,date,venue,match_category,team_1,team_2,team_1_runs,team_1_wickets,team_2_runs,team_2_wickets,result,pom,best_batters,best_bowlers,commentary_line,world_cup_year,host_country
24,7,13,1996-02-25,Colombo (RPS),League-Match,SL,WI,NaN,NaN,NaN,NaN,Sri Lanka won (walkover without a ball bowled),NaN,NaN,NaN,NaN,1996,Sri Lanka
28,11,15,1996-02-26,Patna,League-Match,ZIM,KENYA,45.0,3.0,NaN,NaN,No result,NaN,NaN,NaN,NaN,1996,India
37,20,4,NaN,Colombo (RPS),League-Match,SL,AUS,NaN,NaN,NaN,NaN,Sri Lanka won (walkover without a ball bowled),NaN,NaN,NaN,NaN,1996,Sri Lanka
103,46,14,2019-06-10,Southampton,League-Match,SA,WI,29.0,2.0,NaN,NaN,No result,NaN,"['Q de Kock - 17 runs ', 'HM Amla - 6 runs ']","['SS Cottrell - 2', 'O Thomas - 0']",NaN,2019,England
323,40,19,2011-03-05,Colombo (RPS),League-Match,SL,AUS,146.0,3.0,NaN,NaN,No result,NaN,"['KC Sangakkara - 73 runs ', 'TT Samaraweera -...","['SW Tait - 1', 'B Lee - 1']","19.30 There's no good news, I'm afraid. The ra...",2011,Sri Lanka


Considering null records in `crick_df['team_2_runs']` and Ignoring the abandoned records,

*   Some of the records in the columns `team_1_runs`, `team_2_runs`, `team_1_wickets`, and `team_2_wickets` are missing.

*   Some `team_1_runs`, `team_1_wickets` contains values. But in the result column mention as the **No Result**

Therefore, we cannot get any clear idea through this records.We can remove them


In [11]:
# Filter rows

null_team_2_runs = crick_df[
    crick_df['team_2_runs'].isnull() & ~crick_df['result'].str.contains('abandoned', case=False, na=False)
]

null_team_2_runs.head()

,Unnamed: 0.1,Unnamed: 0,date,venue,match_category,team_1,team_2,team_1_runs,team_1_wickets,team_2_runs,team_2_wickets,result,pom,best_batters,best_bowlers,commentary_line,world_cup_year,host_country
24,7,13,1996-02-25,Colombo (RPS),League-Match,SL,WI,NaN,NaN,NaN,NaN,Sri Lanka won (walkover without a ball bowled),NaN,NaN,NaN,NaN,1996,Sri Lanka
28,11,15,1996-02-26,Patna,League-Match,ZIM,KENYA,45.0,3.0,NaN,NaN,No result,NaN,NaN,NaN,NaN,1996,India
37,20,4,NaN,Colombo (RPS),League-Match,SL,AUS,NaN,NaN,NaN,NaN,Sri Lanka won (walkover without a ball bowled),NaN,NaN,NaN,NaN,1996,Sri Lanka
103,46,14,2019-06-10,Southampton,League-Match,SA,WI,29.0,2.0,NaN,NaN,No result,NaN,"['Q de Kock - 17 runs ', 'HM Amla - 6 runs ']","['SS Cottrell - 2', 'O Thomas - 0']",NaN,2019,England
323,40,19,2011-03-05,Colombo (RPS),League-Match,SL,AUS,146.0,3.0,NaN,NaN,No result,NaN,"['KC Sangakkara - 73 runs ', 'TT Samaraweera -...","['SW Tait - 1', 'B Lee - 1']","19.30 There's no good news, I'm afraid. The ra...",2011,Sri Lanka


In [12]:
records_to_remove = null_team_2_runs.index

# Remove the records
crick_df = crick_df.drop(records_to_remove)

In [13]:
crick_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 518 entries, 0 to 527
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0.1     518 non-null    int64  
 1   Unnamed: 0       518 non-null    int64  
 2   date             355 non-null    object 
 3   venue            518 non-null    object 
 4   match_category   518 non-null    object 
 5   team_1           518 non-null    object 
 6   team_2           518 non-null    object 
 7   team_1_runs      518 non-null    float64
 8   team_1_wickets   518 non-null    float64
 9   team_2_runs      518 non-null    float64
 10  team_2_wickets   518 non-null    float64
 11  result           518 non-null    object 
 12  pom              510 non-null    object 
 13  best_batters     248 non-null    object 
 14  best_bowlers     248 non-null    object 
 15  commentary_line  82 non-null     object 
 16  world_cup_year   518 non-null    int64  
 17  host_country     518 

## Adding New Columns

match_status

In [14]:
crick_df['match_status'] = crick_df['result'].apply(
    lambda x: 'abandoned' if 'match abandoned without a ball bowled' in x.lower() else 'played'
)

winning_team

In [15]:
def get_winning_team(result):
    # Check if the word "won" exists in the result
    if isinstance(result, str) and 'won' in result.lower():
      
        return result.lower().split('won')[0].strip()
    # Return an empty string if "won" is not present
    return ''  

crick_df['winning_team'] = crick_df['result'].apply(get_winning_team)

best_batters and best_bowlers Columns

In [16]:
def extract_best_batters(best_batter_col):
    # Initialize default values in case there are fewer batters
    batter_1_name, batter_1_runs, batter_2_name, batter_2_runs = 'Unknown', 0, 'Unknown', 0
    
    # Check if the column has values (i.e., it's not an empty list)
    if isinstance(best_batter_col, str) and ',' in best_batter_col:
        # Remove unnecessary spaces and split by the comma
        batters = [batter.strip() for batter in best_batter_col.strip("[]").split(',')]
        
       
        if (batters):
           # Extract information for batter 1
            batter_1_name, batter_1_runs = batters[0].split(' - ')

            batter_1_name = batter_1_name.replace("'", "").replace('"', "")
            batter_1_runs = batter_1_runs.replace("runs '", "").replace('runs "', "").strip()
        
            # Extract information for batter 2
            batter_2_name, batter_2_runs = batters[1].split(' - ')

            batter_2_name = batter_2_name.replace("'", "").replace('"', "")
            batter_2_runs = batter_2_runs.replace("runs '", "").replace('runs "', "").strip()

    return batter_1_name, batter_1_runs, batter_2_name, batter_2_runs

# Apply the function to extract the information into new columns
crick_df[['best_batter_1', 'best_batter_1_runs', 'best_batter_2', 'best_batter_2_runs']] = crick_df['best_batters'].apply(
    lambda x: pd.Series(extract_best_batters(x))
)

In [17]:
def extract_best_ballers(best_baller_col):
    # Initialize default values in case there are fewer batters
    baller_1_name, baller_1_wickets, baller_2_name, baller_2_wickets = 'Unknown', 0, 'Unknown', 0
    
    # Check if the column has values (i.e., it's not an empty list)
    if isinstance(best_baller_col, str) and ',' in best_baller_col:
        # Remove unnecessary spaces and split by the comma
        ballers = [baller.strip() for baller in best_baller_col.strip("[]").split(',')]
        
       
        if (ballers):
           # Extract information for baller 1
            baller_1_name, baller_1_wickets = ballers[0].split(' - ')

            baller_1_name = baller_1_name.replace("'", "").replace('"', "")
            baller_1_wickets = int(baller_1_wickets.replace("'", "").replace('"', ""))
        
            # Extract information for baller 2
            baller_2_name, baller_2_wickets = ballers[1].split(' - ')

            baller_2_name = baller_2_name.replace("'", "").replace('"', "")
            baller_2_wickets = int(baller_2_wickets.replace("'", "").replace('"', ""))

    return baller_1_name, baller_1_wickets, baller_2_name, baller_2_wickets

# Apply the function to extract the information into new columns
crick_df[['best_baller_1', 'best_baller_1_wick', 'best_baller_2', 'best_baller_2_wick']] = crick_df['best_bowlers'].apply(
    lambda x: pd.Series(extract_best_ballers(x))
)

In [18]:
# Remove unwanted columns from this dataset

crick_df = crick_df.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'date', 'commentary_line', 'best_batters', 'best_bowlers', 'result'])

## Compute correlation

In [19]:
# crick_df['team_1'].value_counts()
# crick_df['team_2'].value_counts()
# crick_df['host_country'].value_counts()
crick_df['winning_team'].value_counts()


winning_team
australia                                               77
new zealand                                             63
india                                                   63
england                                                 56
pakistan                                                51
south africa                                            49
west indies                                             47
sri lanka                                               40
bangladesh                                              16
                                                        12
zimbabwe                                                11
kenya                                                    7
ireland                                                  7
afghanistan                                              5
netherlands                                              4
match tied (england                                      2
canada                                     

There are some mismatched country types included in above columns. Convert them into Standard type

In [20]:
def naming_countries(country):
    country = country.lower()
    if country == 'australia':
        return 'AUS'
    elif country == 'bangladesh':
        return 'BAN'
    elif country == 'pakistan':
        return 'PAK'
    elif country == 'west indies':
        return 'WI'
    elif country == 'new zealand':
        return 'NZ'
    elif country == 'india':
        return 'IND'
    elif country == 'sri lanka':
        return 'SL'  
    elif country == 'england':
        return 'ENG'
    elif country == 'south africa':
        return 'SA'
    elif country == 'zimbabwe':
        return 'ZIM'
    elif country == 'ireland':
        return 'IRE'
    elif country == 'kenya':
        return 'KENYA'
    elif country == 'afghanistan':
        return 'AFG'
    elif country == 'netherlands':
        return 'NED'
    elif country == 'scotland':
        return 'SCO'
    elif country == 'canada':
        return 'CAN'
    elif country == 'u.a.e':
        return 'UAE'
    else:
        return country.upper()

In [21]:
# Apply the function to the team_1 and team_2 columns

crick_df['team_1'] = crick_df['team_1'].apply(naming_countries)
crick_df['team_2'] = crick_df['team_2'].apply(naming_countries)

In [22]:
# Apply the function to the host_country column

crick_df['host_country'] = crick_df['host_country'].apply(naming_countries)

In [23]:
# Removing mismatched text sections from the winning_team column

crick_df['winning_team'] = crick_df['winning_team'].apply(lambda x: x.split('-')[-1].strip())
crick_df['winning_team'] = crick_df['winning_team'].apply(lambda x: x.split('(')[-1].strip())


# Apply the function to the winning_team columns

crick_df['winning_team'] = crick_df['winning_team'].apply(naming_countries)

### Final Steps in Dataset Cleaning

1. Replace NaN values in the following categorical columns with the placeholder 'Unknown':
    * pom

2. Null Values in `winning_team`:
    * Do not remove null records in the winning_team column, as they correspond to rows where `match_status` is 'abandoned'.


In [24]:
crick_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 518 entries, 0 to 527
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   venue               518 non-null    object 
 1   match_category      518 non-null    object 
 2   team_1              518 non-null    object 
 3   team_2              518 non-null    object 
 4   team_1_runs         518 non-null    float64
 5   team_1_wickets      518 non-null    float64
 6   team_2_runs         518 non-null    float64
 7   team_2_wickets      518 non-null    float64
 8   pom                 510 non-null    object 
 9   world_cup_year      518 non-null    int64  
 10  host_country        518 non-null    object 
 11  match_status        518 non-null    object 
 12  winning_team        518 non-null    object 
 13  best_batter_1       518 non-null    object 
 14  best_batter_1_runs  518 non-null    object 
 15  best_batter_2       518 non-null    object 
 16  best_batter_2

In [25]:
missimg_summary = crick_df.isnull().sum()
missimg_summary

venue                 0
match_category        0
team_1                0
team_2                0
team_1_runs           0
team_1_wickets        0
team_2_runs           0
team_2_wickets        0
pom                   8
world_cup_year        0
host_country          0
match_status          0
winning_team          0
best_batter_1         0
best_batter_1_runs    0
best_batter_2         0
best_batter_2_runs    0
best_baller_1         0
best_baller_1_wick    0
best_baller_2         0
best_baller_2_wick    0
dtype: int64

In [26]:
crick_df['pom'] = crick_df['pom'].fillna('Unknown')

# Replace '' with 'Unknown' in the winning_team column
crick_df['winning_team'] = crick_df['winning_team'].replace('', 'Unknown')


### Checking Duplicates & remove them

In [32]:
crick_df.duplicated().sum()

np.int64(25)

In [33]:
# Extract duplicate rows to duplicates
duplicates = crick_df[crick_df.duplicated(keep=False)]

# Display the duplicate rows
duplicates

,venue,match_category,team_1,team_2,team_1_runs,team_1_wickets,team_2_runs,team_2_wickets,pom,world_cup_year,...,match_status,winning_team,best_batter_1,best_batter_1_runs,best_batter_2,best_batter_2_runs,best_baller_1,best_baller_1_wick,best_baller_2,best_baller_2_wick
0,Manchester,League-Match,CAN,ENG,45.0,0.0,46.0,2.0,Chris Old,1979,...,played,ENG,Unknown,0,Unknown,0,Unknown,0,Unknown,0
6,The Oval,Semi-Final,WI,PAK,293.0,6.0,250.0,0.0,Gordon Greenidge,1979,...,played,WI,Unknown,0,Unknown,0,Unknown,0,Unknown,0
7,Manchester,League-Match,CAN,ENG,45.0,0.0,46.0,2.0,Chris Old,1979,...,played,ENG,Unknown,0,Unknown,0,Unknown,0,Unknown,0
12,The Oval,Semi-Final,WI,PAK,293.0,6.0,250.0,0.0,Gordon Greenidge,1979,...,played,WI,Unknown,0,Unknown,0,Unknown,0,Unknown,0
43,Gwalior,League-Match,WI,IND,173.0,0.0,174.0,5.0,Sachin Tendulkar,1996,...,played,IND,Unknown,0,Unknown,0,Unknown,0,Unknown,0
46,Hyderabad (Deccan),League-Match,ZIM,WI,151.0,9.0,155.0,4.0,Curtly Ambrose,1996,...,played,WI,Unknown,0,Unknown,0,Unknown,0,Unknown,0
49,Gwalior,League-Match,WI,IND,173.0,0.0,174.0,5.0,Sachin Tendulkar,1996,...,played,IND,Unknown,0,Unknown,0,Unknown,0,Unknown,0
53,Hyderabad (Deccan),League-Match,ZIM,WI,151.0,9.0,155.0,4.0,Curtly Ambrose,1996,...,played,WI,Unknown,0,Unknown,0,Unknown,0,Unknown,0
67,Lord's,Final,NZ,ENG,241.0,8.0,241.0,0.0,Ben Stokes,2019,...,played,ENG,BA Stokes,84,JC Buttler,59,CR Woakes,3,LE Plunkett,3
70,Manchester,League-Match,NZ,IND,239.0,8.0,221.0,0.0,Matt Henry,2019,...,played,NZ,RA Jadeja,77,LRPL Taylor,74,MJ Henry,3,B Kumar,3


In [35]:
crick_df.drop_duplicates(inplace=True)

In [36]:
crick_df.shape

(493, 21)

In [30]:
# save the cleaned data to a new csv file

# crick_df.to_csv('./data/crick_df_cleaned.csv')